In [1]:
! pip install pycaret
! pip install feature_engine
! pip install pandas==1.5.3

  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 3.3.2 requires pandas<2.2.0, but you have pandas 2.2.3 which is incompatible.
sktime 0.26.0 requires pandas<2.2.0,>=1.1, but you have pandas 2.2.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is incompatible.
  Using cached pandas-1.5.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
Usi

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/AIGO-2025

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/AIGO-2025


In [3]:
%matplotlib inline
from pycaret.classification import setup, compare_models, create_model, tune_model, finalize_model, \
    evaluate_model, plot_model, predict_model, save_model
import pandas as pd
import os
import numpy as np
from feature_engine.outliers import Winsorizer
from sklearn.impute import SimpleImputer

# Step 1: Read Data

In [4]:
df_train = pd.read_csv("data/training.csv")
df_test = pd.read_csv("data/public_x.csv")

In [5]:
print(df_train.shape)
print(df_test.shape)

(200864, 10214)
(25108, 10213)


In [6]:
df_train.head()

,ID,外資券商_分點進出,外資券商_分點買賣力,外資券商_分點成交力(%),外資券商_分點吃貨比(%),外資券商_分點出貨比(%),外資券商_前1天分點進出,外資券商_前1天分點買賣力,外資券商_前1天分點成交力(%),外資券商_前1天分點吃貨比(%),...,上市加權指數前15天成交量,上市加權指數前16天成交量,上市加權指數前17天成交量,上市加權指數前18天成交量,上市加權指數前19天成交量,上市加權指數前20天成交量,上市加權指數5天成交量波動度,上市加權指數10天成交量波動度,上市加權指數20天成交量波動度,飆股
0,TR-1,1.2227,2.8303,1.2789,0.5707,0.5329,1.2219,1.2686,1.2255,0.5228,...,0.3121,0.1312,0.2415,0.3157,0.4439,0.0533,0.1951,0.1357,0.2388,0
1,TR-2,1.2297,2.8303,1.7995,1.1170,0.5329,1.2313,1.4432,1.2657,1.0660,...,0.9439,0.7407,0.4615,0.4663,0.6618,0.1356,0.4063,0.5800,0.3199,0
2,TR-3,1.3127,1.2826,1.4307,5.0469,4.7141,1.1049,1.2271,1.2514,3.0514,...,0.4925,0.7805,1.2576,0.9081,1.1555,1.4803,0.1924,0.4615,0.7695,0
3,TR-4,1.2219,NaN,1.2413,0.5313,0.5329,1.2258,NaN,1.2238,0.5228,...,1.7776,0.8460,0.8016,0.8309,1.6546,1.5733,0.7886,0.2080,0.7321,0
4,TR-5,0.7988,1.0454,0.8652,1.8842,2.2257,0.7108,1.1333,1.1712,1.5242,...,0.7412,0.8641,0.8693,0.7486,0.2326,0.4883,1.8395,1.4787,0.9049,0


In [12]:
df_train['飆股'].value_counts()

,count
飆股,
0,199394
1,1470


In [ ]:
cols = df_train.columns
for col in df_train.columns:
    if '代號' in col and '買賣超' in col:
      print(col)

# Step 2: Feature Engineering

### Select feature

In [19]:
# import pandas as pd
# import random

# random.seed(123)
# ranking_df = pd.read_csv('./eda_output_polars_v12/feature_ic_original_ranking_full subset.csv')

# top_pct = 0.1
# top_n = int(len(ranking_df) * top_pct)

# # top_features = ranking_df['Feature'].iloc[:top_n].tolist()
# top_features = random.sample(ranking_df['Feature'].tolist(), top_n)
# print(len(top_features))


803


In [7]:
keywords = [
    '外資', '投信', '自營商', '賣賣超', # 籌碼
    '乖離率', '收盤價', '技術指標_K', '技術指標_D', 'MACD',  # 技術
    '月營收', '營業利益', '毛利率' # 基本
    ]
selected_cols = [col for col in df_train.columns if any(kw in col for kw in keywords)]

print(len(selected_cols))

269


In [8]:
df_train_selected = df_train[selected_cols]
df_test_selected = df_test[selected_cols]

### Delete all empty

In [9]:
all_nan_cols = [col for col in df_train_selected.columns if df_train_selected[col].isna().all()]

df_train_selected = df_train_selected.drop(columns=all_nan_cols)
df_test_selected = df_test_selected.drop(columns=all_nan_cols)

print(df_train_selected.shape)
print(df_test_selected.shape)

(200864, 262)
(25108, 262)


### Imputation

In [10]:
# All numerical fields are processed
num_cols = df_train_selected.select_dtypes(include=['int64', 'float64']).columns.tolist()

In [11]:
imputer = SimpleImputer(strategy='mean')
df_train_selected[num_cols] = pd.DataFrame(
    imputer.fit_transform(df_train_selected[num_cols]),
    columns=num_cols,
    index=df_train_selected.index
)
df_test_selected[num_cols] = pd.DataFrame(
    imputer.transform(df_test_selected[num_cols]),
    columns=num_cols,
    index=df_test_selected.index
)

### Outlier Handling

In [12]:
# trim the two tails.
# winsor = Winsorizer(capping_method='iqr', tail='both', fold=1.5, variables=num_cols) # some columns have low variation for method 'iqr'.
winsor = Winsorizer(capping_method='gaussian', tail='both', fold=1.5, variables=num_cols)

df_train_selected = winsor.fit_transform(df_train_selected)
df_test_selected = winsor.transform(df_test_selected)

In [13]:
df_train_final = pd.concat([df_train[['ID']], df_train_selected, df_train[['飆股']]], axis=1)
df_test_final = pd.concat([df_test[['ID']], df_test_selected], axis=1)

In [14]:
print(df_train_final.shape)
print(df_test_final.shape)

(200864, 264)
(25108, 263)


In [15]:
df_train_final.columns

Index(['ID', '外資券商_分點進出', '外資券商_分點買賣力', '外資券商_分點成交力(%)', '外資券商_分點吃貨比(%)',
       '外資券商_分點出貨比(%)', '外資券商_前1天分點進出', '外資券商_前1天分點買賣力', '外資券商_前1天分點成交力(%)',
       '外資券商_前1天分點吃貨比(%)',
       ...
       '上市加權指數前15天收盤價', '上市加權指數前16天收盤價', '上市加權指數前17天收盤價', '上市加權指數前18天收盤價',
       '上市加權指數前19天收盤價', '上市加權指數前20天收盤價', '上市加權指數5天乖離率', '上市加權指數10天乖離率',
       '上市加權指數19天乖離率', '飆股'],
      dtype='object', length=264)

# Step 3: PyCaret Setup

In [16]:
from imblearn.over_sampling import SMOTE

exp_clf = setup(
    data = df_train_final,
    # data = df_train,
    target = '飆股',
    session_id = 123,
    feature_selection = True,        # Enable feature filtering
    n_features_to_select= 0.2,
    fold_strategy='stratifiedkfold',      # Make sure there are two categories for each fold
    ignore_features = ['ID'],
    use_gpu = True,
    fix_imbalance = True,   # Increase the number of minority class
    fix_imbalance_method = SMOTE(),
    transformation = True,         # Do data distribution adjustment (nonlinear conversion)

)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

,Description,Value
0,Session id,123
1,Target,飆股
2,Target type,Binary
3,Original data shape,"(200864, 264)"
4,Transformed data shape,"(339410, 53)"
5,Transformed train set shape,"(279150, 53)"
6,Transformed test set shape,"(60260, 53)"
7,Ignore features,1
8,Numeric features,262
9,Preprocess,True


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Info] Number of positive: 1, number of negative: 1
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset migh

In [17]:
exp_clf.get_config('pipeline')

Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['外資券商_分點進出', '外資券商_分點買賣力',
                                             '外資券商_分點成交力(%)', '外資券商_分點吃貨比(%)',
                                             '外資券商_分點出貨比(%)', '外資券商_前1天分點進出',
                                             '外資券商_前1天分點買賣力',
                                             '外資券商_前1天分點成交力(%)',
                                             '外資券商_前1天分點吃貨比(%)',
                                             '外資券商_前1天分點出貨比(%)', '外資券商_前2天分點進出',
                                             '外資券商_前2天分點買賣力',
                                             '外資券商_前2天分點成交力(%)',
                                             '外資券商_前2天分點吃貨比(%)',
                                             '外...
                                                                                         objective=None,
                                                                                         random_state=None,
                                                                                         reg_alpha=0.0,
                                                                                         reg_lambda=0.0,
                                                                                         subsample=1.0,
                                                                                         subsample_for_bin=200000,
                                                                                         subsample_freq=0),
                                                                importance_getter='auto',
                                                                max_features=52,
                                                                norm_order=1,
                                                                prefit=False,
                                                                threshold=-inf))),
                ('clean_column_names',
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=CleanColumnNames(match='[\\]\\[\\,\\{\\}\\"\\:]+')))],
         verbose=False)

# Step 4: Model training and adjustment

In [18]:
original_model =  compare_models(include=[
    'xgboost',
    # 'et',
    # 'lightgbm',
    # 'dt',
    # 'gbc',
    # 'ada'
])

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9946,0.9777,0.5510,0.6567,0.5982,0.5955,0.5983,69.1170


Processing:   0%|          | 0/9 [00:00<?, ?it/s]

In [19]:
tuned_model = tune_model(original_model, optimize='F1') # Hyperparameter tuning

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9933,0.9764,0.6796,0.5344,0.5983,0.5950,0.5993
1,0.9945,0.9853,0.7184,0.6066,0.6578,0.6550,0.6574
2,0.9927,0.9758,0.5825,0.5000,0.5381,0.5344,0.5360
3,0.9935,0.9711,0.5631,0.5577,0.5604,0.5571,0.5571
4,0.9937,0.9656,0.6275,0.5614,0.5926,0.5894,0.5904
5,0.9942,0.9877,0.6990,0.5854,0.6372,0.6343,0.6368
6,0.9930,0.9807,0.5922,0.5169,0.5520,0.5485,0.5498
7,0.9944,0.9749,0.6408,0.6111,0.6256,0.6228,0.6229
8,0.9950,0.9827,0.6893,0.6455,0.6667,0.6641,0.6645


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[LightGBM] [Info] Number of positive: 125617, number of negative: 125617
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.215258 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 59276
[LightGBM] [Info] Number of data points in the train set: 251234, number of used features: 260
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 125617, number of negative: 125617
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.203139 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 59799
[LightGBM] [Info] Number of data points in the train set: 251234, number of used features: 262
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 125617, num

In [20]:
from sklearn.utils import estimator_html_repr
print(tuned_model.get_params() == original_model.get_params())  # True means the parameters are the same

False


In [21]:
final_model = finalize_model(tuned_model) # Train the model with all data

In [22]:
os.makedirs("model", exist_ok=True)
save_model(final_model, 'model/final_model_20250403')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['外資券商_分點進出', '外資券商_分點買賣力',
                                              '外資券商_分點成交力(%)', '外資券商_分點吃貨比(%)',
                                              '外資券商_分點出貨比(%)', '外資券商_前1天分點進出',
                                              '外資券商_前1天分點買賣力',
                                              '外資券商_前1天分點成交力(%)',
                                              '外資券商_前1天分點吃貨比(%)',
                                              '外資券商_前1天分點出貨比(%)', '外資券商_前2天分點進出',
                                              '外資券商_前2天分點買賣力',
                                              '外資券商_前2天分點成交力(%)',
                                              '外資券商_前2天分點吃貨比(%)',
                                              '外資券商_前2天分點出貨...
                                importance_type=None,
                                interaction_const

# Step 5: Model evaluation

In [29]:
evaluate_model(final_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

# Step 6: Model Prediction

In [ ]:
final_model = load_model('./model/final_model_20250403')

In [25]:
predictions = predict_model(final_model, data=df_test_selected)

In [26]:
predictions

,外資券商_分點進出,外資券商_分點買賣力,外資券商_分點成交力(%),外資券商_分點吃貨比(%),外資券商_分點出貨比(%),外資券商_前1天分點進出,外資券商_前1天分點買賣力,外資券商_前1天分點成交力(%),外資券商_前1天分點吃貨比(%),外資券商_前1天分點出貨比(%),...,上市加權指數前16天收盤價,上市加權指數前17天收盤價,上市加權指數前18天收盤價,上市加權指數前19天收盤價,上市加權指數前20天收盤價,上市加權指數5天乖離率,上市加權指數10天乖離率,上市加權指數19天乖離率,prediction_label,prediction_score
0,1.221900,1.221432,1.2413,0.531300,0.532900,1.225800,1.221389,1.2238,0.52280,0.534300,...,0.2350,0.2142,0.2106,0.2482,0.2772,0.993200,1.088000,1.195100,0,0.9999
1,1.221900,1.221432,1.2413,0.531300,0.532900,1.224300,1.207200,1.2238,0.52280,0.747700,...,2.1812,2.0894,2.0801,2.2595,2.1809,0.616800,0.193700,0.558300,0,1.0000
2,2.733706,1.435600,1.8457,2.712229,2.713048,-0.296378,1.276700,1.3264,2.31780,2.715211,...,1.6711,1.6184,1.6358,1.6796,1.7660,1.153900,0.663000,0.019200,0,1.0000
3,1.221100,-0.032053,0.7311,0.531300,1.051600,1.225800,1.221389,1.2104,0.52280,0.534300,...,0.2202,0.2200,0.1790,0.1886,0.2123,1.047900,0.802200,0.786300,0,1.0000
4,1.568900,1.586100,1.8969,2.481400,1.755300,1.078200,1.301000,1.2881,1.70970,2.032700,...,0.9382,0.9538,0.9505,0.8638,0.9011,-0.280322,-0.280729,-0.278770,0,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25103,1.221900,1.221432,1.2413,0.531300,0.532900,1.225000,1.207200,1.2238,0.52280,0.631100,...,2.0316,1.9386,1.9470,1.9270,1.8446,1.883200,0.946900,0.741000,0,1.0000
25104,1.307300,1.275100,1.3396,2.712229,2.713048,-0.093900,1.213100,1.2345,2.71425,2.715211,...,2.5524,2.5708,2.5030,2.4973,2.4348,0.987500,1.188700,1.612800,0,0.9998
25105,1.204100,-0.032053,0.3484,0.531300,1.440600,1.181100,1.083500,1.1827,0.52280,1.818400,...,0.1760,0.1847,0.1573,0.1994,0.1994,0.566100,0.602400,1.046100,0,1.0000
25106,1.089200,0.686400,0.8777,0.879500,1.239800,1.448800,1.170000,1.2118,1.29270,1.033200,...,2.6849,2.6925,2.6192,2.6323,2.5999,0.337200,0.110300,-0.278770,0,0.9992


In [27]:
if 'ID' in df_test.columns:
    submission = df_test[['ID']].copy()
    submission['飆股'] = predictions['prediction_label']
else:
    submission = predictions[['Label']].copy()
    submission['Id'] = range(1, len(predictions)+1)
    submission = submission[['Id', 'Label']]
    submission.columns = ['Id', 'Category']

os.makedirs("output", exist_ok=True)
submission.to_csv("output/submission_20250403.csv", index=False)

submission.head()

,ID,飆股
0,PU-1,0
1,PU-2,0
2,PU-3,0
3,PU-4,0
4,PU-5,0
